In [253]:
import pandas as pd
import re

# KR operations

In [259]:
def extract_locality_and_street(input_string):
    # Список подстрок, которые могут предшествовать названию населенного пункта
    prefixes = ["г.", "г ", "с.", "с ", "кп ", "к.п.". "Город", "город"]
    locality_keyword = "по адресу:"

    # Начальные значения для города и улицы
    locality = ""
    street = input_string

    locality_index = input_string.find(locality_keyword)
    input_string = input_string[locality_index:]

    # Поиск подстроки-префикса
    for prefix in prefixes:
        index = input_string.find(prefix)
        if index != -1:
            comma_index = input_string.find(',', index)
            locality = input_string[index:comma_index].strip()
            street_start = comma_index + 1
            street = input_string[street_start:].strip()
            break

    return locality, street

In [264]:
KR_files = ["KR on 01.01.2017 unprotected.xlsx", "KR on 01.01.2018 unprotected.xlsx", "KR on 01.01.2019 unprotected.xlsx"]

results = []

for KR_file in KR_files:
    filexlsx = pd.read_excel("train_data/KC_TR_KR/"+KR_file, skiprows=5) # Skip header

    columns_to_drop = [0, 2, 4, 6, 7, 10]
    filexlsx = filexlsx.drop(filexlsx.columns[columns_to_drop], axis=1)

    new_column_names = ["name", "estimated_cost", "approved_year_start", "used_year_end", "expected_year_end"]
    filexlsx.columns = new_column_names

    target_line_startswith = ["Выборочный", "Капитальный", "Новый", "Ремонт"]
    filtered_rows = filexlsx[filexlsx['name'].apply(lambda x: str(x).startswith(tuple(target_line_startswith)) if pd.notna(x) else False)]

    filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))

    results.append(filtered_rows)

/var/folders/hx/c54h6pps2qjgpz_n6c14sk2c0000gp/T/ipykernel_30125/2677059343.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))
/var/folders/hx/c54h6pps2qjgpz_n6c14sk2c0000gp/T/ipykernel_30125/2677059343.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))
/var/folders

In [265]:
results[0]

,name,estimated_cost,used_year_start,approved_year_start,used_year_end,expected_year_end,locality,street
5,Капитальный ремонт подземного сооружения (Лит....,4088.43,160.95,4077.25,3927.48,3927.48,г. Владивосток,"ул. Светланская, д. 71"
8,"Капитальный ремонт здания РКЦ Находка, располо...",19001.91,NaN,1149.83,1143.30,1143.30,г. Находка,"проспект Находкинский, 13"
13,Капитальный ремонт гаража по адресу: Амурская ...,13677.35,819.34,12858.01,12858.01,12858.01,г. Благовещенск,"пер. Святителя Иннокентия, 17"
16,Выборочный капитальный ремонт административног...,18034.12,NaN,1298.34,1298.33,1298.33,г. Тында,"ул. Профсоюзная, 12"
19,Выборочный капитальный ремонт административног...,448.72,NaN,478.98,448.72,448.72,г. Благовещенск,"пер. Св. Иннокентия, 17"
24,Выборочный капитальный ремонт здания гаража на...,4439.86,NaN,199.79,199.79,199.79,г. Петропавловск-Камчатский,"ул. Ломоносова, 2"
29,Выборочный капитальный ремонт сетей теплоснабж...,97.37,NaN,97.37,97.37,97.37,г. Магадан,"ул. Пролетарская, д. 8, строение 1, литера А, ..."
34,Выборочный капитальный ремонт административног...,2344.49,NaN,3281.74,2344.49,2344.49,г. Покровск,"ул. Орджоникидзе, д. 24"
39,"Выборочный капитальный ремонт ""Сооружения комм...",184.94,NaN,184.95,184.94,184.94,,"Выборочный капитальный ремонт ""Сооружения комм..."
42,Выборочный капитальный ремонт в части усиления...,3634.45,325.99,3621.51,3308.46,3308.46,г. Южно-Сахалинск,"Коммунистический проспект, 47 (литер А)"


In [266]:
results[1]

,name,estimated_cost,used_year_start,approved_year_start,used_year_end,expected_year_end,locality,street
5,Капитальный ремонт помещений амбулатории и кро...,10943.56,NaN,670,670.00,670.00,г. Владивосток,"ул. Светланская, 71"
8,"Капитальный ремонт здания РКЦ Находка, располо...",21443.21,1143.30,8817.68,8817.67,8817.67,г. Находка,"проспект Находкинский, д. 13"
11,Выборочный капитальный ремонт ограждения и вор...,5349.85,NaN,5377.36,5349.85,5349.85,г. Уссурийск,"ул. Некрасова, 102"
14,Капитальный ремонт водопровода в гараже на 10 ...,283.66,NaN,283.66,283.66,283.66,г. Уссурийск,"ул. Некрасова, д. 102"
17,Новый объект. Выборочный капитальный ремонт п...,154.41,NaN,154.42,154.41,154.41,г. Уссурийск,"ул. Некрасова, 102"
22,Выборочный капитальный ремонт помещений 3 этаж...,19875.80,NaN,21461.5,19875.80,19875.80,г. Хабаровск,"ул. Муравьева-Амурского, 21"
25,Выборочный капитальный ремонт колодца ВК1 объе...,176.18,NaN,196.5,176.18,176.18,г. Хабаровск,"ул. Калинина, 156"
30,Капитальный ремонт кровли блока А здания по ад...,2506.52,NaN,2506.52,2506.52,2506.52,г. Петропавловск-Камчатский,"просп. К. Маркса, 29/2"
35,Выборочный капитальный ремонт первого этажа во...,2999.90,NaN,2999.9,2999.90,2999.90,г. Южно-Сахалинск,"Коммунистический проспект, 47"
38,"Выборочный капитальный ремонт объекта ""Сооруже...",1699.56,NaN,1699.56,849.78,849.78,,"Выборочный капитальный ремонт объекта ""Сооруже..."


In [267]:
results[2]

,name,estimated_cost,used_year_start,approved_year_start,used_year_end,expected_year_end,locality,street
5,Капитальный ремонт помещений и кровли в пятиэт...,670.00,9170.91,9170.91,9170.90,12.2018,г. Владивосток,"ул. Светланская, 71"
8,"Капитальный ремонт здания РКЦ Находка, располо...",9960.98,11482.32,11482.32,11482.32,5.2018,г.Находка,"проспект Находкинский, д. 13"
11,"Капитальный ремонт здания, расположенного по а...",NaN,375.72,375.72,375.71,10.2019,г.Владивосток,"ул. Калинина, 261"
14,Капитальный ремонт внешнего электроснабжения з...,NaN,290,290,290.00,6.2020,г. Владивосток,"ул. Светланская, д.71, д.73, д.73а"
17,"Капитальный ремонт нежилых помещений (склада),...",NaN,226.46,226.46,226.45,9.2018,г.Владивосток,"ул.Фонтанная, 19"
20,Капитальный ремонт инженерных сетей на объекта...,NaN,1227.79,1227.79,1227.79,7.2018,г. Владивосток,"ул. Пушкинская, 8б"
25,Выборочный капитальный ремонт помещений 2 эта...,NaN,156.65,156.65,0.00,8.2019,г. Хабаровск,"ул. Муравьева-Амурского,21"
28,Выборочный капитальный ремонт аварийных участк...,NaN,175.56,175.56,140.46,12.2018,г. Хабаровск,участок находится примерно в 166 м по направле...
33,Выборочный капитальный ремонт здания гаража на...,199.79,3397.65,3397.65,3397.65,8.2018,г. Петропавловск-Камчатский,"ул. Ломоносова, 2"
36,Выборочный капитальный ремонт системы хозяйств...,NaN,490.53,490.53,490.53,8.2018,г. Петропавловск-Камчатский,"просп. К.Маркса, 29/2"


# New version sheets

In [279]:
KR_files = ["KR on 01.01.2020.xlsx", "KR on 01.01.2021.xlsx", "KR on 01.01.2022.xlsx"]

for KR_file in KR_files:
    filexlsx = pd.read_excel("train_data/KC_TR_KR/"+KR_file, skiprows=2) # Skip header

    columns_to_drop = [0, 2, 4, 5, 7, 9, 10, 12, 13, 14, 15, 16, 17]
    filexlsx = filexlsx.drop(filexlsx.columns[columns_to_drop], axis=1)

    new_column_names = ["name", "estimated_cost", "approved_year_start", "used_year_end", "expected_year_end"]
    filexlsx.columns = new_column_names

    target_line_startswith = ["Выборочный", "Капитальный", "Новый", "Ремонт"]
    filtered_rows = filexlsx[filexlsx['name'].apply(lambda x: str(x).startswith(tuple(target_line_startswith)) if pd.notna(x) else False)]

    filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))

    results.append(filtered_rows)


/var/folders/hx/c54h6pps2qjgpz_n6c14sk2c0000gp/T/ipykernel_30125/913290712.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))
/var/folders/hx/c54h6pps2qjgpz_n6c14sk2c0000gp/T/ipykernel_30125/913290712.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))
/var/folders/h

In [280]:
results[3]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
11,Капитальный ремонт здания убежища (лит. А) Дал...,9437312.0,1800131,0,0,г. Владивосток,"ул. Семеновская, 29а"
14,Капитальный ремонт внешнего электроснабжения з...,6706610.0,6416610,6416609.38,6416609.38,г. Владивосток,"ул. Светланская, д.71, д.73, д.73а"
17,"Капитальный ремонт здания, расположенного по а...",83550896.0,1469476,1469475.8,1469475.8,г. Владивосток,"ул. Светланская, 71"
20,Капитальный ремонт нежилых помещений и помещен...,30232722.0,1653670,0,0,г. Владивосток,"ул. Светланская, д.73"
23,Выборочный капитальный ремонт помещения бокс-...,12529120.0,1429234,0,0,г. Владивосток,"ул. Светланская, 71"
26,"Капитальный ремонт здания, расположенного по а...",4529888.0,1653938,1654433.44,1654433.44,с. Вольно-Надеждинское,"ул. Пушкина, д. 59"
29,"Выборочный капитальный ремонт РКЦ, пристройки ...",15492993.0,14736188,0,0,г. Уссурийск,"ул. Некрасова, д.102"
32,Выборочный капитальный ремонт гаража РКЦ на 2 ...,2014976.0,1877466,908618.4,908618.4,г. Уссурийск,"ул. Некрасова, д.102"
35,Выборочный капитальный ремонт нежилых помещен...,2875630.0,263600,0,0,г. Владивосток,"ул. Светланская, д.73"
38,Выборочный капитальный ремонт здания Централь...,3942549.0,258003,258002.34,258002.34,г. Владивосток,"ул. Океанский проспект, д.34"


In [281]:
results[4]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
11,Выборочный капитальный ремонт здания Централь...,3942549.0,1836158,0,0,г. Владивосток,"ул. Океанский проспект, д.34"
14,Выборочный капитальный ремонт здания Централь...,97782.0,97782,97782,97782,г. Владивосток,"Океанский проспект, д.34 в части замены дверно..."
17,Выборочный капитальный ремонт нежилых помещен...,2907500.0,2907500,235594.9,235594.9,г. Владивосток,"ул. Светланская, д.73"
20,Выборочный капитальный ремонт помещения бокс-...,12784075.0,12309375,2308489.84,2308489.84,г. Владивосток,"ул. Светланская, 71"
23,Капитальный ремонт здания убежища (лит. А) Дал...,9465229.0,9163939,9198951.17,9198951.17,г. Владивосток,"ул. Семеновская, 29а"
26,Капитальный ремонт нежилых помещений и помещен...,31288369.0,30776847,15126513.98,15126513.98,г. Владивосток,"ул. Светланская, д.73"
29,"Капитальный ремонт здания, расположенного по а...",4924748.0,2981808,2981311.49,2981311.49,с. Вольно-Надеждинское,"ул. Пушкина, д. 59"
32,"Выборочный капитальный ремонт РКЦ, пристройки ...",16313168.0,15556363,5325763.61,5325763.61,г. Уссурийск,"ул. Некрасова, д.102"
35,Выборочный капитальный ремонт гаража РКЦ на 2 ...,1946333.0,895205,895204.38,895204.38,г. Уссурийск,"ул. Некрасова, д.102"
38,Выборочный капитальный ремонт гаража РКЦ на 10...,1534438.0,1483529,1483528.28,1483528.28,г. Уссурийск,"ул. Некрасова, д.102"


In [282]:
results[5]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
11,Выборочный капитальный ремонт здания Централь...,4415994.0,4157991,3871001.7,3871001.7,г. Владивосток,"ул. Океанский проспект, д.34"
14,Выборочный капитальный ремонт нежилого здания ...,4243675.0,4243675,139445.85,139445.85,г. о. Владивостокский,"г. Владивосток, пр-кт Океанский, д. 34 в части..."
17,Выборочный капитальный ремонт помещения бокс-...,12475698.0,9692508,3909529.02,3909529.02,г. Владивосток,"ул. Светланская, 71"
20,Выборочный капитальный ремонт нежилых помещени...,2907500.0,2671905,945594.46,945594.46,г. Владивосток,"ул. Светланская, д.73"
23,Капитальный ремонт нежилых помещений и помещен...,31288369.0,15650333,11776819.09,11776819.09,г. Владивосток,"ул. Светланская, д.73"
26,Выборочный капитальный ремонт нежилого здания ...,4074527.0,153689,153688.78,153688.78,,Выборочный капитальный ремонт нежилого здания ...
29,"Выборочный капитальный ремонт РКЦ, пристройки ...",16107760.0,1212791,1212790.63,1212790.63,г. Уссурийск,"ул. Некрасова, д.102"
32,Выборочный капитальный ремонт с созданием авто...,57279.0,57279,57278.4,57278.4,г.о. Владивостокский,"г. Владивосток, пр-кт Океанский, д. 34"
35,Выборочный капитальный ремонт помещения № 14 в...,1212904.0,1212904,1212903.56,1212903.56,,Выборочный капитальный ремонт помещения № 14 в...
39,Капитальный ремонт с перепланировкой помещений...,43837947.0,999621,999620.92,999620.92,,Капитальный ремонт с перепланировкой помещений...
